# Churn Prediction
### Redes neurais para análise da saída de clientes
O projeto abaixo visa o desenvolvimento de uma rede neural para identificar e auxiliar na previsao e análise da perda de clientes (churn) de uma base de dados economicos encontrados no Kaggle, [diretamente nesse link](https://www.kaggle.com/datasets/mervetorkan/churndataset).

Nosso Conjunto de dados é baseado nos seguintes atributos:

* <b> RowNumber:</b> número da linha (int);
* <b> CustomerID:</b> número do Id (int);
* <b> Surname:</b> Sobrenome da pessoa (string);
* <b> CreditScore:</b> avaliação do crédito (int);
* <b> Geography:</b> Região de origem (string);
* <b> Gender:</b> genero (string);
* <b> Age:</b> idade da pessoa (int);
* <b> Tenure:</b> Quantidade de posses de valor que a pessoa tem (int);
* <b> Balance:</b> balanço da conta (float64);
* <b> NumOfPRoducts:</b> numero de produtos comprados (int);
* <b> HasCrCard:</b> a pessoa possui ou nao cartao de credito (int - 0 não possui e 1 possui);
* <b> IsActiveMember: </b> é um clinte ativo (int - 0 para sim, 1 para nao);
* <b> Exilted:</b> se o cliente saiu ou nao da empresa (int - 0 para sim e 1 para nao).

Agora iremos fazer as importações necessárias para o projeto.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Dropout


2025-05-14 11:43:43.438362: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 11:43:43.447088: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-14 11:43:43.470950: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747233823.510755   26498 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747233823.523510   26498 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747233823.553756   26498 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Carregando os dados e observando se eles estão carregados corretamente.

In [2]:
dados = pd.read_csv('src/dados/churn.csv')
dados.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


## Modificando os dados para treinar a rede
Agora iremos observar os dados e efetuar as modificações necessárias dentro do conjunto para treinar a nossa rede.

In [3]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Como nossos dados vieram de um dataset limpo e conciso do Kaggle nosso dataset nao possui valores nulos e nem problemas que exigem tratamento aprofundado, uma vez que nosso objetivo é realmente a prática de desenvolvimento de uma rede neural.

Uma observação inicial importante é sobre os atributos que temos nos dados do dataset, uma vez que as três primeiras colunas, <b>RowNumber, CustomerId e Surname não são necessárias na nossa análise e nao precisam fazer parte do nosso conjunto de dados </b>. Na verdade o CustomerId e o Surname podemos considerar dados senciveis (principalmente as duas ultimas se combinadas com algum outro conjunto de dados nao tratados) e ainda expor os clientes de um dataset real sem a necessidade, indo contra a lei LGPD. Entretanto nao iremos focar nesses dados de forma excessiva uma vez que sao dados liberados no kaggle e nosso objetivo é o estudo das redes neurais.

In [4]:
dados = dados.drop(columns=['RowNumber','CustomerId', 'Surname'])
dados.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.0,1,1,1,101348.88,1


Agora que efetuamos a exclusao das nossas colunas desnecessárias podemos dar continuidade a nossa rede neural. Como observamos na descrição no topo do nosso arquivo o objetivo aqui é efetuar o modelo para descobrir e analisar a saida dos clientes dos dados que estamos analisando, coluna essa denominada como 'Exited', logo, essa coluna será o nosso atributo de resposta. Para separarmos os dados podemos então ja dividir nossos dados em X e y para começarmos a trabalhar os dados para a rede.

### Padronizando os dados e dividindo em treino e teste

Efetuaremos a separação do nosso atributo de objetivo, que será a saída ou nao do cliente em relação aos nossos dados. Para isso vamos trabalhar os dados que precisam de transformação categorica, sendo o  genero, que pode ser 'Male' (homem) ou 'Female' (mulher), e o 'Geograhphy' (que trás os paises do conjunto de dados), que podem ser 'France' (França), 'Spain' (Espanha) e 'Germany'(Alemanha).

Para efetuar essa transformação iremos aplicar o 'columnTransformer' para aplicar a transformação em diferentes colunas e one_hot_enconding, pois esse comando vai transformar os valores em diferentes colunas e aplicar como zeros para negativos e um para positivos dos nosso dados. Por exemplo, existira uma coluna 'Male' para definir se a pessoa em questão é homem ou mulher, para homem sera um na coluna e para mulher sera zero, o mesmo ocorrendo na coluna 'Female', isso é util para trasnformar os dados que eram categoricos em dados que a rede neural entenda de forma efetiva e consiga aprender com os dados. Entao tem-se:
* ColumnTransformer - permite a transformação em multiplas colunas do dataset;
* transformers - a lista de transformações que iremos aplicar;
* drop='first' - retira a primeira coluna após a transformação, que evita a multicolinearidade.
* o remainder é aplicado para evitar a modificação das colunas que não foram citadas.

Falando especificamente sobre a multicolinearidade, o 'drop='first' ajuda a evitar redundancia de informação, aumentando o desempenho do nosso modelo.

In [5]:
column_transformer = ColumnTransformer(
  transformers=[('onehoteconding', OneHotEncoder(drop='first'), ['Geography', 'Gender'])],
                  remainder='passthrough')

X = column_transformer.fit_transform(dados.drop(columns='Exited'))
#aplicando a transformalçao para nosso X, onde será nossos dados com exceção do atributo target

y = dados['Exited']
#nosso atributo target

Após a modificações dentro da nossa transformação aplicamos o 'columns_transformer' dentro dos nossos dados sendo os atributos passado para o valor de X, dados esses que serão os dados utilizados para treinar nossa rede neural. O dado de objetivo é o 'Exited', a saída da rede neural é a resposta da saída ou nao do cliente, entrando como valor y, que será nosso objetivo a ser encontrado.

Agora iremos padronizar os numeros e na sequencia efetuar o treinamento. A padronização é importante uma vez que os dados apresenta valores desproporcionais entre eles no próprio atributo e entre os atributos, entao essa modificação permite um melhor desempenho do nosso modelo ML.

In [6]:
X = StandardScaler().fit_transform(X)

Efetuada a transformação podemos agora dividir nossos dados em treino e teste, que em resumo ira utilizar os dados de treino para treinar nossa rede neural e os dados de teste para efetuar os testes de acuracia da rede, sendo na proporção de 80% para teste e 20% para treino.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)

## Sobre a rede a ser desenvolvida
Alguns pontos relevantes sobre a rede neural que será desenvolvida:
* Serão adicionadas camadas relacionadas com o Dense, onde cada camada intermediaria terá 16 neuronios, apenas um na camada de saida;
* A função Relu para ativação também será aplicada nas camadas de inicio e do meio, buscando linearizar os valores entre zero e um;
* A função de ativação na saída será sigmoide, uma vez que essa função é a mais indicada para uma resposta do tipo binaria;
* Nosso termo de analise de perda será a 'binary_crossentropy', pois também é a mais indicada para casos binarios;
* O otimizador é o Adam, para tentar diminuir as perdas;
* A metrica de acerto sera a acuracia.

In [8]:
classifier = Sequential()

#adicionar as camadas
classifier = Sequential()
classifier.add(Dense(units=16, kernel_initializer='he_uniform', activation='relu', input_dim=X_train.shape[1]))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=16, kernel_initializer='he_uniform', activation='relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units=1, activation='sigmoid'))
#ultima camada é apenas um neuronio, sendo sigmoid, por ser binario nossa resposta, a expectativa é uma melhor resposta

classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(X_train, y_train, batch_size=10, epochs=30)


/home/fabio/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-14 11:43:51.718887: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6917 - loss: 0.6647
Epoch 2/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7873 - loss: 0.5044
Epoch 3/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8035 - loss: 0.4663
Epoch 4/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8053 - loss: 0.4608
Epoch 5/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8085 - loss: 0.4407
Epoch 6/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8148 - loss: 0.4317
Epoch 7/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8234 - loss: 0.4147
Epoch 8/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8263 - loss: 0.4152
Epoch 9/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8180 - loss: 0.4194
Epoch 10/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8302 - loss: 0.4033
Epoch 11/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8272 - loss: 0.4011
Epoch 12/30
800/800 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step

Nosso modelo conseguiu uma acuracia de 85% aproximadamente e uma perda de 0.36, o treinamento foi bem efetuado e conseguiu bons valores dentro do conjunto de dados. Importante ressalta que podemos ver a melhoria a cada nova rodada de treinamento e aprendizado dentro da nossa rede. Vale ressaltar que algumas das funções de ativaçoes e inicializadores merecem ser citadas de forma mais detalhada e serão comentados em um adendo ao fim desse arquivo.

## Prevendo os dados
Agora que temos nosso modelo treinado aplicamos o modelo nos dados de teste.

In [9]:
y_previsto = classifier.predict(X_test)
y_previsto = (y_previsto >0.5)

z_previsto = np.array([f"{value[0] * 100:.2f}%" for value in y_previsto])
print(z_previsto)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
['0.00%' '0.00%' '0.00%' ... '100.00%' '0.00%' '0.00%']


Agora aplicamos nossa confusion matrix para encontrar nossa acuracia.

In [10]:
confusion_matrix = pd.DataFrame(confusion_matrix(y_test,y_previsto))
confusion_matrix

,0,1
0,1570,37
1,249,144


In [11]:
total = y_previsto.shape[0]

true_negatives = confusion_matrix.values[0][0]
true_positives = confusion_matrix.values[1][1]

acuracia = (true_negatives+true_positives)/total

print(f'A acuracia da rede neural foi de {round(acuracia,2)*100}%')

A acuracia da rede neural foi de 86.0%


### Salvando o modelo
Agora iremos salvar o modelo para finalizar o projeto

In [12]:
classifier.save('churn_classifier_model.h5')

Caso queira importar o modelo, segue o codigo abaixo.

In [13]:
"""from tensorflow.keras.models import load_model

#Carrega o modelo salvo
modelo_carregado = load_model('churn_classifier_model.h5')"""

"from tensorflow.keras.models import load_model\n\n#Carrega o modelo salvo\nmodelo_carregado = load_model('churn_classifier_model.h5')"

## Adendo sobre o Kernel_initializer e o Relu
Como demonstrado o nosso modelo foi desenvolvido baseado em um problema de classificação binária, onde os valores podem assumir apenas a saida ou nao do cliente. Com aplicação de camadas Dense e com inicialização de pesos (kernel_initializer).

No kernel_initializer utiliza o he_uniform para inicializar, com a aplicação do ReLu esse he_uniform evita a explosão do gradiente dentro da nossa rede, melhorando o desempenho do nosso modelo, iniciando já com uma distribuição uniforme de pesos.

O ReLu (rectified Linear Unit) evita problemas com o gradiente (gradient vanishing) sendo muito bem aplicado em camadas ocultas, conseguindo o bom aprendizado do modelo de forma nao linear.

O sigmoid é aplicada na camada final pois é indicado em saidas binarias do conjunto, sendo interpretado como uma porcentagem de acerto na saida, pois seus valores variam entre zero e um.
